### 第九章 EM算法

In [20]:
import numpy as np
from typing import Tuple
def em_3_coin(y: np.ndarray, pi: float, p: float, q:float, max_iter: int=1000) -> Tuple[float, float, float]:
    """
    三硬币模型
    :param y 样本数据
    :param pi 初始值pi
    :param p 初始值p
    :param q 初始值q
    :param max_iter 最大迭代次数
    :return pi 收敛值pi
    :return p 收敛值p
    :return q 收敛值q
    """
    for i in range(max_iter):
        mu = (
            pi * np.power(p, y) * np.power(1-p, 1-y) /
            (pi * np.power(p, y) * np.power(1-p, 1-y) + (1-pi) * np.power(q, y) * np.power(1-q, 1-y))
            )
        new_pi = np.mean(mu)
        new_p = np.sum(y * mu) / np.sum(mu)
        new_q = np.sum(y * (1 - mu)) / np.sum(1 - mu)

        pi, p, q = new_pi, new_p, new_q
    return new_pi, new_p, new_q


In [24]:
y = np.array([1, 1, 0, 1, 0, 0, 1, 0, 1, 1])
print(em_3_coin(y, 0.5, 0.5, 0.5, max_iter=200))
print(em_3_coin(y, 0.4, 0.6, 0.7, max_iter=200))
print(em_3_coin(y, 0.46, 0.55, 0.67, max_iter=2000))

(np.float64(0.5), np.float64(0.6), np.float64(0.6))
(np.float64(0.40641711229946526), np.float64(0.5368421052631581), np.float64(0.6432432432432431))
(np.float64(0.4618628351139197), np.float64(0.5345950037850103), np.float64(0.6561346417857334))


In [28]:
def norm(x: np.ndarray, mu: float, sigma: float) -> np.ndarray:
    """
    维高斯分布概率密度函数
    :param x: 输入数据（一维数组）
    :param mu: 均值
    :param sigma: 标准差
    :return: 概率密度值
    """
    coeff = 1.0 / (np.sqrt(2 * np.pi) * sigma)
    exponent = -0.5 * ((x - mu) / sigma) ** 2
    pdf = coeff * np.exp(exponent)
    return pdf


def em_2_gaussian(y: np.ndarray, max_iter: int=1000, tol: float=1e-6 ) -> Tuple[float, float, float, float, float]:
    """
    两分量GMM的EM算法实现
    :param y: 观测数据
    :param max_iter: 最大迭代次数
    :param tol: 收敛阈值
    :return: alpha, mu1, var1, mu2, var2
    """
    # 初始化参数
    n = len(y)
    alpha = 0.5
    y = np.sort(y)
    mu1 = y[:2].mean() 
    mu2 = y[2:].mean() 
    var1 = y[:2].var()
    var2 = y[2:].var()
    
    # 迭代EM
    for _ in range(max_iter):
        alpha_prev, mu1_prev, var1_prev, mu2_prev, var2_prev = alpha, mu1, var1, mu2, var2
        
        # E
        p1 = alpha * norm(y, mu1, np.sqrt(var1))
        p2 = (1 - alpha) * norm(y, mu2, np.sqrt(var2))
        gamma1 = p1 / (p1 + p2)
        gamma2 = p2 / (p1 + p2)
        
        # M 
        alpha = np.sum(gamma1) / n
        mu1 = np.sum(gamma1 * y) / np.sum(gamma1)
        mu2 = np.sum(gamma2 * y) / np.sum(gamma2)
        var1 = np.sum(gamma1 * (y - mu1)**2) / np.sum(gamma1)
        var2 = np.sum(gamma2 * (y - mu2)**2) / np.sum(gamma2)
        
        param_diff = np.abs([alpha - alpha_prev, mu1 - mu1_prev, var1 - var1_prev,
                             mu2 - mu2_prev, var2 - var2_prev]).max()
        if param_diff < tol:
            break
    
    return alpha, mu1, var1, mu2, var2

y = np.array([-67, -48, 6, 8, 14, 16, 23, 24, 28, 29, 41, 49, 56, 60, 75])
result = em_2_gaussian(y)
print([float(x) for x in result])

[0.13317228120710103, -57.51107685634485, 90.24987767714728, 32.98488732315319, 429.4583427156817]
